In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException,ElementClickInterceptedException

import json
import numpy as np
import pandas as pd
import io
from time import sleep
import datetime
import csv
from tqdm import tqdm
import re

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.google.com")

[WDM] - Downloading: 100%|██████████| 6.78M/6.78M [01:38<00:00, 72.3kB/s]


In [2]:
data = pd.read_excel('../seedlist.xlsx')
data.head()

,competitor_name,competitor_link,competitor_twitter
0,Rise at Seven,https://riseatseven.com/work/,https://twitter.com/RiseAtSeven
1,Reboot,https://www.rebootonline.com/placements/,NaN
2,Kaizen,https://kaizen.co.uk/work/,https://twitter.com/kaizen_agency
3,Motive PR,https://www.motivepr.co.uk/work,https://twitter.com/motivepr
4,Don't Panic,https://dontpanic.agency/,NaN


In [2]:
delay = 2

def waiting_until_full(delay,type):
    try:
        element_present = EC.presence_of_all_elements_located((By.XPATH, type))
        WebDriverWait(driver, delay).until(element_present)
    except TimeoutException:
        pass

In [3]:
seedlist_competitor = pd.read_excel('seedlist.xlsx','competitors')
seedlist_competitor

,competitor_name,competitor_link,competitor_twitter
0,Rise at Seven,https://riseatseven.com/work/,https://twitter.com/RiseAtSeven
1,Reboot,https://www.rebootonline.com/placements/,NaN
2,Kaizen,https://kaizen.co.uk/work/,https://twitter.com/kaizen_agency
3,Motive PR,https://www.motivepr.co.uk/work,https://twitter.com/motivepr
4,Don't Panic,https://dontpanic.agency/,NaN
5,Verve Search,https://www.vervesearch.com/casestudies/,NaN
6,Priceonomics,https://priceonomics.com/,https://twitter.com/priceonomics
7,Neomam,https://neomam.com/portfolio,NaN
8,Fractl,https://www.frac.tl/work/content-marketing-cas...,https://twitter.com/fractlagency
9,Digitaloft,https://digitaloft.co.uk/success-stories/,NaN


In [4]:
seedlist_client = pd.read_excel('seedlist.xlsx','clients')
seedlist_client.head()

,client_link,client_name
0,https://www.electronicshub.org/insights/,electronicshub
1,https://www.budgetdirect.com.au/,budgetdirect
2,https://www.poundstopocket.co.uk/,poundstopocket
3,https://www.homeadvisor.com/,homeadvisor
4,https://www.angi.com/,angi


In [11]:
#riseatseven

def riseatseven():
    pr_camps = pd.DataFrame(columns=['Name', 'Link', 'Source', 'Published Date'])

    for comp_link in seedlist_competitor['competitor_link']:
        if 'riseatseven' in comp_link:
            try:
                driver.get(comp_link)
                waiting_until_full(delay, '//*[@id="gatsby-focus-wrapper"]/main/div[5]/div[4]')
                digital_pr = driver.find_element(By.XPATH, '//*[@id="gatsby-focus-wrapper"]/main/div[5]/div[4]')
                lists = digital_pr.find_elements(By.TAG_NAME,'a')
                links = [lists[i].get_attribute('href') for i in range(len(lists))]
                for link in links:
                    driver.get(link)
                    waiting_until_full(delay, '/html/body')
                    sleep(2)
                    names = driver.find_element(By.XPATH,'//*[@id="gatsby-focus-wrapper"]/main/div[1]/div[2]/div[1]/div/div/div/div/h3[2]').text
                    text = driver.find_element(By.XPATH, '//*[@id="gatsby-focus-wrapper"]/main/div[3]/div/div[3]/div')
                    total_links_xpath = text.find_elements(By.TAG_NAME, 'a')
                    total_links =  list({total_links_xpath[i].get_attribute('href') for i in range(len(total_links_xpath))})
                    p_date = driver.find_element(By.XPATH, '//*[@id="gatsby-focus-wrapper"]/main/div[3]/div/div[1]/div/div[3]/h4').text
                    url = driver.current_url
                    # ex_link = [l for l in total_links if 'weareyard' not in l]
                    links_final = [l for l in total_links if 'riseatseven' not in l]
                    pr_camps = pd.concat([pr_camps, pd.DataFrame.from_records([{'Name': names, 'Link': links_final, 'Source': url, 'Published Date': p_date}])])
                    sleep(2)

                print("Riseatseven's total campaigns are: {}".format(len(pr_camps)))
                # df = pd.DataFrame(list(pr_camps.items()),columns=['Name','Links'])
                # df['source'] = 'riseatseven'
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace('[',''))
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace(']',''))
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace("'",''))
            except Exception as e:
                # pr_camps = pd.DataFrame()
                continue
    return pr_camps

In [34]:
#reboot
def reboot():
    pr_camps = pd.DataFrame(columns=['Name', 'Link', 'Source', 'Published Date'])

    for comp_link in seedlist_competitor['competitor_link']:
        if 'reboot' in comp_link:
            try:
                driver.get(comp_link)
                waiting_until_full(delay, '/html/body/script[2]')
                json_raw = json.loads(driver.find_element(By.XPATH, '/html/body/script[2]').get_attribute('innerHTML')[15:-1])
                for i in json_raw:
                    for j in range(len(i['list'])):
                        name = i['list'][j]['campaign']['name'].title()
                        link = i['list'][j]['url_link']
                        source = driver.current_url
                        p_date = i['list'][j]['campaign']['start_date']
                # pr_camps = {i['list'][j]['campaign']['name'].title(): i['list'][j]['url_link'] for i in json_raw for j in range(len(i['list']))}
                        pr_camps = pd.concat([pr_camps, pd.DataFrame.from_records([{'Name': name, 'Link': link, 'Source': source, 'Published Date': p_date}])])
                print("Rebootonline's total campaigns are: {}".format(len(pr_camps)))
            except Exception as e:
                print('1')
                # pr_camps = {}
                continue
            # df = pd.DataFrame(list(pr_camps.items()),columns=['Name','Links'])
            pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace('[',''))
            pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace(']',''))
            pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace("'",''))
    return pr_camps

In [10]:
#motivepr
def motivepr():
    pr_camps = pd.DataFrame(columns=['Name', 'Link', 'Source', 'Published Date'])

    for comp_link in seedlist_competitor['competitor_link']:
        if 'motivepr' in comp_link:
            try:
                driver.get(comp_link)
                waiting_until_full(delay, '/html/body/div[5]')
                sleep(2)
                digital_pr = driver.find_element(By.XPATH, '/html/body/div[5]/div/div')
                lists = digital_pr.find_elements(By.TAG_NAME,'a')
                links = [lists[i].get_attribute('href') for i in range(len(lists))]
                for link in links:
                    driver.get(link)
                    waiting_until_full(delay, '//*[@id="content"]/section[2]')
                    sleep(2)
                    names = driver.find_element(By.TAG_NAME,'h1').text
                    source = driver.current_url
                    p_date = 'unavailable'
                    text = driver.find_element(By.XPATH, '/html/body/div[11]/div/div/div[1]')
                    total_links_xpath = text.find_elements(By.TAG_NAME, 'a')
                    total_links =  list({total_links_xpath[i].get_attribute('href') for i in range(len(total_links_xpath))})
                    links_final = [l for l in total_links if 'motivepr' not in l]
                    pr_camps = pd.concat([pr_camps, pd.DataFrame.from_records([{'Name': names, 'Link': links_final, 'Source': source, 'Published Date': p_date}])])
                    sleep(2)
                driver.back()
                sleep(2)
                print("MotivePR's total campaigns: {}".format(len(pr_camps)))
            except Exception as e:
                continue

    # df = pd.DataFrame(list(pr_camps.items()),columns=['Name','Links'])
    # df['source'] = 'motivepr'
    pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace('[',''))
    pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace(']',''))
    pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace("'",''))
            # print(df)
    return pr_camps

In [15]:
#dontpanic
def dontpanic():
    pr_camps = pd.DataFrame(columns=['Name', 'Link', 'Source', 'Published Date'])

    for comp_link in seedlist_competitor['competitor_link']:
        if 'dontpanic' in comp_link:
            driver.get(comp_link)
            waiting_until_full(delay, '//*[@id="work_items"]')
            digital_pr = driver.find_element(By.XPATH, '//*[@id="work_items"]')
            camps = [camp.get_attribute('href') for camp in digital_pr.find_elements(By.TAG_NAME, 'a')]
            try:
                for i in camps:
                    driver.get(i)
                    waiting_until_full(delay, '//*[@id="new_page_holder_inner"]/div/div[1]/div[1]')
                    sleep(10)
                    name = driver.find_element(By.TAG_NAME, 'h1').text
                    links = driver.find_element(By.CLASS_NAME,'link').find_element(By.TAG_NAME, 'a').get_attribute('href')
                    source = driver.current_url
                    p_date = 'unavailable'
                    pr_camps = pd.concat([pr_camps, pd.DataFrame.from_records([{'Name': name, 'Link': links, 'Source': source, 'Published Date': p_date}])])
                    driver.back()
                    waiting_until_full(delay, '//*[@id="work_items"]')
                    sleep(10)
            except Exception as e:
                continue
    print("Dontpanic's total campaigns are: {}".format(len(pr_camps)))
    pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace('[',''))
    pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace(']',''))
    pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace("'",''))
    return pr_camps

In [23]:
#vervesearch
def vervesearch():
    pr_camps = pd.DataFrame(columns=['Name', 'Link', 'Source', 'Published Date'])

    for comp_link in seedlist_competitor['competitor_link']:
        if 'vervesearch' in comp_link:
            driver.get(comp_link)
            waiting_until_full(delay, '/html/body/main/section[1]/div/ul')
            digital_pr = driver.find_element(By.XPATH, '/html/body/main/section[1]/div/ul')
            camps = [camp.get_attribute('href') for camp in digital_pr.find_elements(By.TAG_NAME, 'a')]
            try:
                for camp in camps:
                    names = camp.split('/')[-2].replace('-',' ').title()
                    driver.get(camp)
                    waiting_until_full(delay, '/html/body/main/section[2]/div/a')
                    link = driver.find_elements(By.CLASS_NAME, 'button')[0].get_attribute('href')
                    source = camp
                    p_date = 'unavailable'
                    pr_camps = pd.concat([pr_camps, pd.DataFrame.from_records([{'Name': names, 'Link': link, 'Source': source, 'Published Date': p_date}])])
                    driver.back()
                    waiting_until_full(delay, '/html/body/main/section[1]/div/ul')
                    sleep(3)
            except Exception as e:
                continue
    print("Vervesearch's total campaigns are: {}".format(len(pr_camps)))
    pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace('[',''))
    pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace(']',''))
    pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace("'",''))
    return pr_camps

In [37]:
#neomam
def neomam():
    pr_camps = pd.DataFrame(columns=['Name', 'Link', 'Source', 'Published Date'])

    for comp_link in seedlist_competitor['competitor_link']:
        if 'neomam' in comp_link:
            try:
                driver.get(comp_link)
                waiting_until_full(delay, '//*[@id="__layout"]/div/div/div[2]')
                for i in range(10):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    sleep(2)
                waiting_until_full(delay, '//*[@id="__layout"]/div/div/div[2]')
                sleep(2)
                digital_pr = driver.find_elements(By.LINK_TEXT,'see it live')
                for i, camp in enumerate(digital_pr):
                    camps = camp.get_attribute('href')
                    names = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div/div[2]/div/div[{}]/div[2]/p[1]/b'.format(i+1)).text
                    source = driver.current_url
                    p_date = 'unavailable'
                    pr_camps = pd.concat([pr_camps, pd.DataFrame.from_records([{'Name': names, 'Link': camps, 'Source': source, 'Published Date': p_date}])])
                print("Neomam's total campaigns: {}".format(len(pr_camps)))
            except Exception as e:
                continue

            pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace('[',''))
            pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace(']',''))
            pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace("'",''))
    return pr_camps

In [54]:
#fractl
def fractl():
    pr_camps = {}

    for comp_link in seedlist_competitor['competitor_link']:
        if 'frac' in comp_link:
            try:
                driver.get(comp_link)
                waiting_until_full(delay, '/html/body/section[2]/div')
                sleep(2)
                pr_camps = driver.find_element(By.XPATH,'/html/body/section[2]/div')
                all_links = pr_camps.find_elements(By.TAG_NAME, 'a')
                links = list({link.get_attribute('href') for link in all_links})
                for link in links:
                    driver.get(link)
                    waiting_until_full(delay, '/html/body')
                    sleep(2)
                    names = driver.find_element(By.TAG_NAME,'h1').text
                    text = driver.find_element(By.XPATH, '/html/body')
                    total_links_xpath = text.find_elements(By.TAG_NAME, 'a')
                    total_links =  list({total_links_xpath[i].get_attribute('href') for i in range(len(total_links_xpath))})
                    # ex_link = [l for l in total_links if 'weareyard' not in l]
                    pr_camps[names] = [l for l in total_links if 'frac' not in l]
                    sleep(2)
                driver.back()
                sleep(2)
                print("Fractl's total campaigns: {}".format(len(pr_camps)))
            except Exception as e:
                continue
    df = pd.DataFrame(list(pr_camps.items()),columns=['Name','Links'])
    df['source'] = 'fractl'
    df['Links'] = df['Links'].map(lambda x: str(x).replace('[',''))
    df['Links'] = df['Links'].map(lambda x: str(x).replace(']',''))
    df['Links'] = df['Links'].map(lambda x: str(x).replace("'",''))
    return df

In [39]:
#digitaloft
def digitaloft():
    pr_camps = pd.DataFrame(columns=['Name', 'Link', 'Source', 'Published Date'])
    
    for comp_link in seedlist_competitor['competitor_link']:
        if 'digitaloft' in comp_link:
            try:
                driver.get(comp_link)
                waiting_until_full(delay, '//*[@id="page"]/div[4]/div/div')
                sleep(2)
                digital_pr = driver.find_element(By.XPATH,'//*[@id="page"]/div[4]/div/div')
                camps = digital_pr.find_elements(By.TAG_NAME, 'a')
                links = list({camp.get_attribute('href') for camp in camps})
                names = [link.split('/')[-2].replace('-',' ').title() for link in links]
                pr_camps = {names[i]: links[i] for i in range(len(names))}
                print("Digitaloft's total campaigns: {}".format(len(pr_camps)))
            except Exception as e:
                continue

            df = pd.DataFrame(list(pr_camps.items()),columns=['Name','Links'])
            df['source'] = 'https://digitaloft.co.uk/success-stories/'
            df['Publish Date'] = 'unavailable'
            df['Links'] = df['Links'].map(lambda x: str(x).replace('[',''))
            df['Links'] = df['Links'].map(lambda x: str(x).replace(']',''))
            df['Links'] = df['Links'].map(lambda x: str(x).replace("'",''))
    return df

In [45]:
#weareyard
def weareyard():
    pr_camps = pd.DataFrame(columns=['Name', 'Link', 'Source', 'Published Date'])

    for comp_link in seedlist_competitor['competitor_link']:
        if 'weareyard' in comp_link:
            try:
                driver.get(comp_link)
                waiting_until_full(delay, '//*[@id="content"]')
                sleep(2)
                digital_pr = driver.find_elements(By.CLASS_NAME, 'casestudy_content')
                links = [digital_pr[i].find_element(By.TAG_NAME,'a').get_attribute('href') for i in range(len(digital_pr))]
                # names = [digital_pr[i].find_element(By.TAG_NAME,'h3').text for i in range(len(digital_pr))]
                for link in links:
                    driver.get(link)
                    waiting_until_full(delay, '//*[@id="content"]/section[2]')
                    sleep(2)
                    names = driver.find_element(By.TAG_NAME,'h1').text
                    text = driver.find_element(By.XPATH, '//*[@id="content"]/section[2]')
                    total_links_xpath = text.find_elements(By.TAG_NAME, 'a')
                    total_links =  list({total_links_xpath[i].get_attribute('href') for i in range(len(total_links_xpath))})
                    # ex_link = [l for l in total_links if 'weareyard' not in l]
                    links_final = [l for l in total_links if 'weareyard' not in l]
                    source = driver.current_url
                    p_date = 'unavailable'
                    pr_camps = pd.concat([pr_camps, pd.DataFrame.from_records([{'Name': names, 'Link': links_final, 'Source': source, 'Published Date': p_date}])])
                    sleep(2)
                driver.back()
                sleep(2)
                print("Weareyard's total campaigns: {}".format(len(pr_camps)))
            except Exception as e:
                continue

    pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace('[',''))
    pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace(']',''))
    pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace("'",''))
    return pr_camps

In [46]:
weareyard()

Weareyard's total campaigns: 3


,Name,Link,Source,Published Date
0,Building brand with a viral digital PR campaign,https://weareyard.com/case-studies/yard-celeb-...,https://weareyard.com/case-studies/yard-celeb-...,unavailable
0,Demonstrating digital PR creativity for VisitS...,https://weareyard.com/case-studies/visitscotla...,https://weareyard.com/case-studies/visitscotla...,unavailable
0,Creating captivating campaigns for Wealthify,https://weareyard.com/case-studies/wealthify,https://weareyard.com/case-studies/wealthify,unavailable


# Client Lists

In [64]:
#electronicshub

def electronicshub():
    pr_camps = pd.DataFrame(columns=['Name', 'Link', 'Source', 'Published Date'])

    for client_link in seedlist_client['client_link']:
        if 'electronicshub' in client_link:
            try:
                driver.get(client_link)
                waiting_until_full(delay, '/html/body/div[1]/section[2]/div/div[1]')
                digital_pr = driver.find_element(By.XPATH, '/html/body/div/section[2]/div/div[1]/div/div[1]/div')
                a_ids = digital_pr.find_elements(By.TAG_NAME, 'article')
                list_id = [id.get_attribute('id') for id in a_ids]
                lists = digital_pr.find_elements(By.TAG_NAME,'a')
                for i in range(len(lists)):
                    links = driver.find_element(By.XPATH, '//*[@id="{}"]/div/section/div/div[2]/div/div[2]/div/h2/a'.format(list_id[i])).get_attribute('href')
                    names = driver.find_element(By.XPATH, '//*[@id="{}"]/div/section/div/div[2]/div/div[2]/div/h2/a'.format(list_id[i])).text
                    source = driver.current_url
                    p_date = driver.find_element(By.XPATH, '//*[@id="{}"]/div/section/div/div[2]/div/div[4]/div/ul/li/span'.format(list_id[i])).text
                    pr_camps = pd.concat([pr_camps, pd.DataFrame.from_records([{'Name': names, 'Link': links, 'Source': source, 'Published Date': p_date}])])
                print("electronicshub's total campaigns are: {}".format(len(pr_camps)))
                # df = pd.DataFrame(list(pr_camps.items()),columns=['Name','Links'])
                # df['source'] = 'https://www.electronicshub.org/insights/'
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace('[',''))
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace(']',''))
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace("'",''))
                # pr_camps = pr_camps.iloc[2:,:]
            except Exception as e:
                continue 
    return pr_camps

In [10]:
#budgetdirect

def budgetdirect():
    pr_camps = {}

    for client_link in seedlist_client['client_link']:
        if 'budgetdirect' in client_link:
            try:
                main_links = ['https://www.budgetdirect.com.au/car-insurance/articles.html', 'https://www.budgetdirect.com.au/travel-insurance/articles.html',
                        'https://www.budgetdirect.com.au/home-contents-insurance/articles.html', 'https://www.budgetdirect.com.au/life-insurance/articles.html']
                for main_link in main_links:
                    driver.get(main_link)
                    waiting_until_full(delay, '/html/body/div[4]/div[2]/div/div/div/div/div/div/div/div/div')
                    digital_pr = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div/div/div/div/div/div/div/div/div')
                    lists = digital_pr.find_elements(By.TAG_NAME,'a')
                    links = [link.get_attribute('href') for link in lists if link.get_attribute('class') == 'column__link']                  
                    texts = digital_pr.find_elements(By.TAG_NAME, 'h3')
                    names = [text.text for text in texts]
                    pr_camps = {names[i]: links[i] for i in range(len(links))}
                    print("budgetdirect's total campaigns are: {}".format(len(pr_camps)))
                    df = pd.DataFrame(list(pr_camps.items()),columns=['Name','Link'])

            except Exception as e:
                continue
    df['Source'] = 'https://www.budgetdirect.com.au/'
    df['Published Date'] = 'unavailable'
    df['Link'] = df['Link'].map(lambda x: str(x).replace('[',''))
    df['Link'] = df['Link'].map(lambda x: str(x).replace(']',''))
    df['Link'] = df['Link'].map(lambda x: str(x).replace("'",''))
    return df

In [16]:
#businessbacker

def businessbacker():
    pr_camps = pd.DataFrame(columns=['Name', 'Link', 'Source', 'Published Date'])

    for client_link in seedlist_client['client_link']:
        if 'businessbacker' in client_link:
            try:
                for i in range(1,22):
                    driver.get(client_link + 'page/{}/'.format(i))
                    waiting_until_full(delay, '/html/body/div[2]/main/section[4]/div/div[1]/div[1]/div')
                    sleep(5)
                    digital_pr = driver.find_element(By.XPATH, '/html/body/div[2]/main/section[4]/div/div[1]/div[1]/div')
                    source = driver.current_url
                    for j in range(1,11):
                        links = driver.find_element(By.XPATH, '/html/body/div[2]/main/section[4]/div/div[1]/div[1]/div/div[{}]/div/p/a'.format(j)).get_attribute('href')
                        names = driver.find_element(By.XPATH, '/html/body/div[2]/main/section[4]/div/div[1]/div[1]/div/div[{}]/div/p/a'.format(j)).text
                        p_date =  driver.find_element(By.XPATH, '/html/body/div[2]/main/section[4]/div/div[1]/div[1]/div/div[{}]/div/time'.format(j)).text
                        pr_camps = pd.concat([pr_camps, pd.DataFrame.from_records([{'Name': names, 'Link': links, 'Source': source, 'Published Date': p_date}])])
                    # main_df = main_df.append(df, ignore_index=True)
                print("businessbacker's total campaigns are: {}".format(len(pr_camps)))
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace('[',''))
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace(']',''))
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace("'",''))
            except Exception as e:
                continue
    return  pr_camps

In [17]:
businessbacker()

,Name,Link,Source,Published Date
0,What Is Loan Stacking?,https://www.businessbacker.com/blog/what-is-lo...,https://www.businessbacker.com/blog/most-recent/,"October 10, 2022"
0,What To Do if Your Application Is Declined,https://www.businessbacker.com/blog/applicatio...,https://www.businessbacker.com/blog/most-recent/,"August 25, 2022"
0,4 Ways Multiple Financing Options Can Help Gro...,https://www.businessbacker.com/blog/financing-...,https://www.businessbacker.com/blog/most-recent/,"August 18, 2022"
0,How to Dispute Errors on Your Credit Report,https://www.businessbacker.com/blog/dispute-er...,https://www.businessbacker.com/blog/most-recent/,"June 15, 2022"
0,Does Cash Flow Affect My Business Loan Applica...,https://www.businessbacker.com/blog/does-cash-...,https://www.businessbacker.com/blog/most-recent/,"April 6, 2022"
...,...,...,...,...
0,Quick Loans Can Help Your Business Find the Ca...,https://www.businessbacker.com/blog/quick-loan...,https://www.businessbacker.com/blog/most-recen...,"October 15, 2013"
0,What Small Business Loans Best Suit Your Company?,https://www.businessbacker.com/blog/best-small...,https://www.businessbacker.com/blog/most-recen...,"October 8, 2013"
0,How to Become a Restaurateur!,https://www.businessbacker.com/blog/how-to-bec...,https://www.businessbacker.com/blog/most-recen...,"October 7, 2013"
0,Don’t Hire A Business Coach Who Can’t Answer T...,https://www.businessbacker.com/blog/dont-hire-...,https://www.businessbacker.com/blog/most-recen...,"October 1, 2013"


In [131]:
#expedia
# need fixing
def expedia():
    pr_camps = {}
    c = 0
    for client_link in seedlist_client['client_link']:
        if 'expedia' in client_link:
            try:
                driver.get(client_link)
                waiting_until_full(delay, '//*[@id="ag-main"]/div/div[4]')
                digital_pr = driver.find_element(By.XPATH, '//*[@id="ag-main"]/div/div[4]')
                button = digital_pr.find_element(By.XPATH, '//*[@id="ag-main"]/div/div[4]/button')
                while c <= 20:
                    button.click()
                    sleep(delay)
                    c += 1
                lists = digital_pr.find_elements(By.TAG_NAME,'a')
                links = [link.get_attribute('href') for link in lists]
                texts = digital_pr.find_elements(By.CLASS_NAME, 'ag-archive-post-title')
                names = [text.text for text in texts]
                pr_camps = {names[i]: links[i] for i in range(len(links))}
                print("expedia's total campaigns are: {}".format(len(pr_camps)))
                df = pd.DataFrame(list(pr_camps.items()),columns=['Name','Links'])
                df['source'] = 'expedia'
                df['Links'] = df['Links'].map(lambda x: str(x).replace('[',''))
                df['Links'] = df['Links'].map(lambda x: str(x).replace(']',''))
                df['Links'] = df['Links'].map(lambda x: str(x).replace("'",''))
            except Exception as e:
                continue 
    return df

In [21]:
#homes

def homes():
    pr_camps = pd.DataFrame(columns=['Name', 'Link', 'Source', 'Published Date'])

    for client_link in seedlist_client['client_link']:
        if 'homes.com' in client_link:
            try:
                for i in range(1,21):
                    driver.get(client_link + 'page/{}/'.format(i))
                    waiting_until_full(delay, '//*[@id="primary"]/div[2]/div')
                    sleep(5)
                    posts = driver.find_elements(By.TAG_NAME, 'article')
                    post_ids = [id.get_attribute('id') for id in posts]
                    source = driver.current_url
                    for post in post_ids:
                        links = driver.find_element(By.XPATH, '//*[@id="{}"]/div/div[1]/a'.format(post)).get_attribute('href')
                        names = driver.find_element(By.XPATH, '//*[@id="{}"]/div/div[1]/a/h2'.format(post)).text
                        p_date = driver.find_element(By.XPATH, '//*[@id="{}"]/div/div[2]/span[2]'.format(post)).text
                        pr_camps = pd.concat([pr_camps, pd.DataFrame.from_records([{'Name': names, 'Link': links, 'Source': source, 'Published Date': p_date}])])
                    # main_df = main_df.append(df, ignore_index=True)
                print("homes's total campaigns are: {}".format(len(pr_camps)))
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace('[',''))
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace(']',''))
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace("'",''))
            except Exception as e:
                continue
    return  pr_camps

In [31]:
#surfshark

def surfshark():
    pr_camps = pd.DataFrame(columns=['Name', 'Link', 'Source', 'Published Date'])
    for client_link in seedlist_client['client_link']:
        if 'surfshark' in client_link:
            try:
                for i in range(1,6):
                    if i == 1:
                        driver.get(client_link)
                        waiting_until_full(delay, '//*[@id="__next"]/div/main/div[2]/div[2]/div')
                        sleep(5)
                        source = driver.current_url
                        for j in range(1,7):
                            names = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main/div[2]/div[2]/div/div[{}]/a/div/h3'.format(j)).text                                   
                            links = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main/div[2]/div[2]/div/div[{}]/a'.format(j)).get_attribute('href')
                            p_date = 'unavailable'
                            pr_camps = pd.concat([pr_camps, pd.DataFrame.from_records([{'Name': names, 'Link': links, 'Source': source, 'Published Date': p_date}])])
                    else:
                        driver.get(client_link + '/{}'.format(i))
                        waiting_until_full(delay, '//*[@id="__next"]/div/main/div[2]/div/div')
                        sleep(5)
                        for j in range(1,7):
                            names = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main/div[2]/div/div/div[{}]/a/div/h3'.format(j)).text
                            links = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main/div[2]/div/div/div[{}]/a'.format(j)).get_attribute('href')
                            p_date = 'unavailable'
                            pr_camps = pd.concat([pr_camps, pd.DataFrame.from_records([{'Name': names, 'Link': links, 'Source': source, 'Published Date': p_date}])])
                    waiting_until_full(delay, '//*[@id="__next"]/div/main/div[2]/div[2]/div')
                    sleep(5)

                    # main_df = main_df.append(df, ignore_index=True)
                print("surfshark's total campaigns are: {}".format(len(pr_camps)))
                pr_camps['source'] = 'surfshark'
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace('[',''))
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace(']',''))
                pr_camps['Link'] = pr_camps['Link'].map(lambda x: str(x).replace("'",''))
            except Exception as e:
                continue
    return  pr_camps

In [32]:
surfshark()

,Name,Link,Source,Published Date
0,Global Data Breach Map,https://surfshark.com/research/data-breach-mon...,https://surfshark.com/research,unavailable
0,Internet Shutdown Tracker,https://surfshark.com/research/internet-censor...,https://surfshark.com/research,unavailable
0,Government Surveillance Report,https://surfshark.com/user-data-surveillance-r...,https://surfshark.com/research,unavailable
0,Data Vulnerability Thermometer,https://surfshark.com/research/data-breach-impact,https://surfshark.com/research,unavailable
0,How people see their privacy in 2022,https://surfshark.com/attitude-on-privacy,https://surfshark.com/research,unavailable
0,Data breach statistics 2022 Q1,https://surfshark.com/blog/data-breach-statist...,https://surfshark.com/research,unavailable
0,Data breach statistics by country in 2021,https://surfshark.com/blog/data-breach-statist...,https://surfshark.com/research,unavailable
0,The data flows: How private are popular period...,https://surfshark.com/blog/period-track-app-da...,https://surfshark.com/research,unavailable
0,Cybersecurity in Ukraine and other former Sovi...,https://surfshark.com/blog/cybersecurity-ukrai...,https://surfshark.com/research,unavailable
0,Children and online risks: global statistics,https://surfshark.com/research/cybersecurity-f...,https://surfshark.com/research,unavailable


In [35]:
#stltraining

def stltraining():
    pr_camps = {}
    for client_link in seedlist_client['client_link']:
        if 'stl-training' in client_link:
            try:
                driver.get(client_link)
                waiting_until_full(delay, '//*[@id="beststl_maincontent"]/div[2]/section/div')
                digital_pr = driver.find_element(By.XPATH, '//*[@id="beststl_maincontent"]/div[2]/section/div')
                lists = digital_pr.find_elements(By.TAG_NAME,'a')
                links = list({lists[i].get_attribute('href') for i in range(len(lists))})
                names = [lists[i].text for i in range(len(lists)) if lists[i].text != '']
                pr_camps = {names[i]: links[i] for i in range(len(links))}
                print("stltraining's total campaigns are: {}".format(len(pr_camps)))
                df = pd.DataFrame(list(pr_camps.items()),columns=['Name','Links'])
                df['Source'] = 'https://www.stl-training.co.uk/sharing/'
                df['Published Date'] = 'unavailable'
                df['Links'] = df['Links'].map(lambda x: str(x).replace('[',''))
                df['Links'] = df['Links'].map(lambda x: str(x).replace(']',''))
                df['Links'] = df['Links'].map(lambda x: str(x).replace("'",''))
                df = df.iloc[2:,:]
            except Exception as e:
                continue 
    return df

In [36]:
stltraining()

stltraining's total campaigns are: 101


,Name,Links,Source,Published Date
2,► Power BI Overview [Video],https://www.stl-training.co.uk/sharing/13-how-...,https://www.stl-training.co.uk/sharing/,unavailable
3,► What do I do with my feet when presenting? [...,https://www.stl-training.co.uk/sharing/how-to-...,https://www.stl-training.co.uk/sharing/,unavailable
4,► How do I keep good eye contact when presenti...,https://www.stl-training.co.uk/sharing/techniq...,https://www.stl-training.co.uk/sharing/,unavailable
5,► What is good video call etiquette? [Video],https://www.stl-training.co.uk/sharing/the-gro...,https://www.stl-training.co.uk/sharing/,unavailable
6,► Professional Development Virtual Classroom S...,https://www.stl-training.co.uk/sharing/how-to-...,https://www.stl-training.co.uk/sharing/,unavailable
...,...,...,...,...
96,A well-trained I.T. team: Your company's secre...,https://www.stl-training.co.uk/sharing/how-to-...,https://www.stl-training.co.uk/sharing/,unavailable
97,The History of Microsoft,https://www.stl-training.co.uk/sharing/excel-a...,https://www.stl-training.co.uk/sharing/,unavailable
98,What's in and what's out in Office 2013?,https://www.stl-training.co.uk/sharing/14-exce...,https://www.stl-training.co.uk/sharing/,unavailable
99,Just How Big Is Microsoft? - Microsoft trainin...,https://www.stl-training.co.uk/sharing/futurep...,https://www.stl-training.co.uk/sharing/,unavailable


# Adhoc

In [6]:
driver = webdriver.Chrome(r'D:\TKMedia\chromedriver_win32-chromedriver.exe')
driver.set_page_load_timeout(20)
driver.maximize_window()

C:\Users\Admin\AppData\Local\Temp\ipykernel_20916\4241941079.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'D:\TKMedia\chromedriver_win32-chromedriver.exe')


In [7]:
driver.get('https://defiyield.app/rekt-database')

In [49]:
final = pd.DataFrame(columns=['Project', 'Chain', 'Category', 'Issue', 'Value Lost', 'Date'])
for i in range(1,201):
    try:
        project = driver.find_element(By.XPATH, '//*[@id="pushable"]/div/main/section/div[4]/div[2]/div[{}]/div[1]/div[1]/div/a/span'.format(i)).text
        chain = driver.find_element(By.XPATH,'//*[@id="pushable"]/div/main/section/div[4]/div[2]/div[{}]/div[1]/div[2]'.format(i)).text
        category = driver.find_element(By.XPATH,'//*[@id="pushable"]/div/main/section/div[4]/div[2]/div[{}]/div[1]/div[3]'.format(i)).text
        issue = driver.find_element(By.XPATH,'//*[@id="pushable"]/div/main/section/div[4]/div[2]/div[{}]/div[1]/div[4]/span'.format(i)).text
        value_lost = driver.find_element(By.XPATH,'//*[@id="pushable"]/div/main/section/div[4]/div[2]/div[{}]/div[1]/div[5]/span'.format(i)).text
        date = driver.find_element(By.XPATH,'//*[@id="pushable"]/div/main/section/div[4]/div[2]/div[{}]/div[1]/div[6]'.format(i)).text
        final = pd.concat([final, pd.DataFrame.from_records([{'Project': project, 'Chain': chain, 'Category': category, 'Issue': issue, 'Value Lost': value_lost, 'Date': date}])])
    except Exception as e:
        project = driver.find_element(By.XPATH, '//*[@id="pushable"]/div/main/section/div[4]/div[2]/div[{}]/div[1]/div[1]/div/a/span'.format(i)).text
        chain = driver.find_element(By.XPATH,'//*[@id="pushable"]/div/main/section/div[4]/div[2]/div[{}]/div[1]/div[2]'.format(i)).text
        category = driver.find_element(By.XPATH,'//*[@id="pushable"]/div/main/section/div[4]/div[2]/div[{}]/div[1]/div[3]'.format(i)).text
        issue = driver.find_element(By.XPATH,'//*[@id="pushable"]/div/main/section/div[4]/div[2]/div[{}]/div[1]/div[4]/span'.format(i)).text
        value_lost = 'missing'
        date = driver.find_element(By.XPATH,'//*[@id="pushable"]/div/main/section/div[4]/div[2]/div[{}]/div[1]/div[6]'.format(i)).text
        final = pd.concat([final, pd.DataFrame.from_records([{'Project': project, 'Chain': chain, 'Category': category, 'Issue': issue, 'Value Lost': value_lost, 'Date': date}])])
final.head()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="pushable"]/div/main/section/div[4]/div[2]/div[170]/div[1]/div[1]/div/a/span"}
  (Session info: chrome=110.0.5481.78)
Stacktrace:
Backtrace:
	(No symbol) [0x00E437D3]
	(No symbol) [0x00DD8B81]
	(No symbol) [0x00CDB36D]
	(No symbol) [0x00D0D382]
	(No symbol) [0x00D0D4BB]
	(No symbol) [0x00D43302]
	(No symbol) [0x00D2B464]
	(No symbol) [0x00D41215]
	(No symbol) [0x00D2B216]
	(No symbol) [0x00D00D97]
	(No symbol) [0x00D0253D]
	GetHandleVerifier [0x010BABF2+2510930]
	GetHandleVerifier [0x010E8EC1+2700065]
	GetHandleVerifier [0x010EC86C+2714828]
	GetHandleVerifier [0x00EF3480+645344]
	(No symbol) [0x00DE0FD2]
	(No symbol) [0x00DE6C68]
	(No symbol) [0x00DE6D4B]
	(No symbol) [0x00DF0D6B]
	BaseThreadInitThunk [0x76AE7D69+25]
	RtlInitializeExceptionChain [0x775ABB9B+107]
	RtlClearBits [0x775ABB1F+191]


In [51]:
final.to_clipboard()